In [1]:
%load_ext autoreload
%autoreload 2
from typing import *

In [2]:
from utils import load_input

# Part 1

As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are as follows:
```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
      9   + 4 * 5 + 6
         13   * 5 + 6
             65   + 6
                 71
```
Parentheses can override this order; for example, here is what happens if parentheses are added to form 1 + (2 * 3) + (4 * (5 + 6)):
```
1 + (2 * 3) + (4 * (5 + 6))
1 +    6    + (4 * (5 + 6))
     7      + (4 * (5 + 6))
     7      + (4 *   11   )
     7      +     44
            51
```
Here are a few more examples:
```
2 * 3 + (4 * 5) becomes 26.
5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437.
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240.
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632.
```
Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?



In [3]:
import operator

ops = {
    '+': operator.add,
    '*': operator.mul,
}

In [4]:
def parse_expr(expr: str, pntr: int):
    left, op = None, None
    while pntr < len(expr):
        v = expr[pntr]
        if v == " ":
            pntr += 1
        elif v in ops:
            op = ops[v]
            pntr += 1
        # ignore parens for a sec
        elif v == ")":
            return left, pntr+1
        elif v == "(":
            res, pntr = parse_expr(expr, pntr+1)
            if left is not None:
                left = op(left, res)
            else:
                left = res
        else:
            pntr += 1
            if left is not None:
                left = op(left, int(v))
            else:
                left = int(v)
    return left, pntr

In [5]:
parse_expr("1 + 2 * 3 + 4 * 5 + 6", 0)

(71, 21)

In [6]:
parse_expr("1 + (2 * 3) + (4 * (5 + 6))", 0)

(51, 27)

In [7]:
tests = """2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2""".splitlines()
for test in tests:
    print(parse_expr(test, 0))

(26, 15)
(437, 27)
(12240, 41)
(13632, 47)


In [8]:
data = load_input(18)

In [9]:
sum(v for v, _ in (parse_expr(d, 0) for d in data))

75592527415659

# Part 2

You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are now as follows:
```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
  3   *   7   * 5 + 6
  3   *   7   *  11
     21       *  11
         231
```
Here are the other examples from above:

```
1 + (2 * 3) + (4 * (5 + 6)) still becomes 51.
2 * 3 + (4 * 5) becomes 46.
5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445.
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060.
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340.
```
What do you get if you add up the results of evaluating the homework problems using these new rules?



In [21]:
def place_left_paren(expr, idx):
    expr = list(expr)
    non_ints = "()+*"
    n_rparens = 0
    n_lparens = 0
    while idx > 0:
        v = expr[idx]
        if v not in non_ints and (n_lparens == n_rparens):
            expr.insert(idx, "(")
            return expr
        elif v == ")":
            n_rparens += 1
            idx -= 1
        elif v == "(":
            n_lparens +=1
            if n_lparens == n_rparens:
                expr.insert(idx, "(")
                return expr
            else:
                idx -= 1
        else:
            idx -= 1
    expr.insert(0, "(")
    return expr
            
def place_right_paren(expr, idx):
    expr = list(expr)
    non_ints = "()+*"
    n_rparens = 0
    n_lparens = 0
    while idx < len(expr):
        v = expr[idx]
        if v not in non_ints and (n_lparens == n_rparens):
            expr.insert(idx+1, ")")
            return expr
        elif v == "(":
            n_lparens += 1
            idx += 1
        elif v == ")":
            n_rparens +=1
            if n_lparens == n_rparens:
                expr.insert(idx+1, ")")
                return expr
            else:
                idx += 1
        else:
            idx += 1
    expr.insert(len(expr), ")")
    return expr

def preprocess(expr: str) -> str:
    start = 0
    offset = 0
    non_ints = "()+*"
    while (idx:=expr.find("+", start)) != -1:
        #print(f"{expr=}")
        #print(f"{idx=}")
        l_idx = idx - 1
        expr = "".join(place_left_paren(expr, l_idx))
        r_idx = idx + 2 # we inserted a paren
        expr = "".join(place_right_paren(expr, r_idx))
        offset += 2
        start = idx + 2
        #print(f"{start=}")
        #print(f"{expr=}")
    return expr            

In [22]:
preprocess("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2".replace(" ", ""))

'(((((2+4)*9)*(((6+9)*(8+6))+6))+2)+4)*2'

In [23]:
for test in tests:
    print(parse_expr(preprocess(test.replace(" ", "")), 0))

(46, 11)
(1445, 21)
(669060, 29)
(23340, 39)


In [26]:
sum(v for v, _ in (parse_expr(preprocess(d.replace(" ", "")), 0) for d in data))

360029542265462